In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_10%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,NaN,98.0,1.0
1,0.0,13.0,3250.0,28.0,1.0
2,1.0,16.0,4000.0,NaN,1.0
3,2.0,20.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,23.0,2.0,500.0,NaN,0.0
744,NaN,2.0,NaN,52.0,0.0
745,NaN,NaN,NaN,62.0,0.0
746,39.0,1.0,250.0,NaN,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              107
Frequency (times)                              78
Monetary (c.c. blood)                          94
Time (months)                                  95
whether he/she donated blood in March 2007      0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 374
persentase : 10.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              14.304813
Frequency (times)                             10.427807
Monetary (c.c. blood)                         12.566845
Time (months)                                 12.700535
whether he/she donated blood in March 2007     0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,9)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.0,50.0,0.0,98.0
1,0.0,13.0,3250.0,28.0
2,1.0,16.0,4000.0,0.0
3,2.0,20.0,5000.0,45.0
4,1.0,24.0,6000.0,77.0
...,...,...,...,...
743,23.0,2.0,500.0,0.0
744,0.0,2.0,0.0,52.0
745,0.0,0.0,0.0,62.0
746,39.0,1.0,250.0,0.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 379


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)

Kolom : 1
[2.0, 4.0, 1.0, 2.0, 14.0, 21.0, 12.0, 2.0, 0.0]
mean :  6.444444444444445
standar deviasi :  6.961977865698019
fuzzy :  [(2.0, 0.8156673940222168), (4.0, 0.9402269889392392), (1.0, 0.7365705464397709), (2.0, 0.8156673940222168), (14.0, 0.554974290319463), (21.0, 0.11244048588233138), (12.0, 0.7273429300235916), (2.0, 0.8156673940222168), (0.0, 0.6515628237112178)]
fuzzy :  None
pembilang :  28.250488294614566
penyebut :  6.170120247382266
rata-rata tertimbang :  4.578596066519143
 
[4.0, 0.0, 4.0, 9.0, 11.0, 9.0, 11.0, 2.0, 4.0]
mean :  6.0
standar deviasi :  3.8297084310253524
fuzzy :  [(4.0, 0.8725376293860391), (0.0, 0.2931281251357148), (4.0, 0.8725376293860391), (9.0, 0.7358077750283618), (11.0, 0.4264823228174336), (9.0, 0.7358077750283618), (11.0, 0.4264823228174336), (2.0, 0.5796110577942652), (4.0, 0.8725376293860391)]
fuzzy :  None
pembilang :  34.25682472071505
penyebut :  5.814932266779688
rata-rata tertimbang :  5.891182072132114
 
[11.0, 1.0, 5.0, 6.0, 11.0, 2.

[23.0, 0.0, 8.0, 3.0, 14.0, 23.0, 11.0, 0.0, 11.0]
mean :  10.333333333333334
standar deviasi :  8.219218670625303
fuzzy :  [(23.0, 0.30502032204820717), (0.0, 0.4537476323575653), (8.0, 0.9605090500741785), (3.0, 0.6716720712867531), (14.0, 0.9052933341381185), (23.0, 0.30502032204820717), (11.0, 0.99671627065351), (0.0, 0.4537476323575653), (11.0, 0.99671627065351)]
fuzzy :  None
pembilang :  58.331888060982095
penyebut :  6.048442905617614
rata-rata tertimbang :  9.644116505887022
 
[2.0, 3.0, 4.0, 14.0, 16.0, 4.0, 2.0, 2.0, 2.0]
mean :  5.444444444444445
standar deviasi :  5.187830013301668
fuzzy :  [(2.0, 0.8022058841221206), (3.0, 0.8949408391444998), (4.0, 0.9619840110974578), (14.0, 0.2567322954366284), (16.0, 0.12621865653037945), (4.0, 0.9619840110974578), (2.0, 0.8022058841221206), (2.0, 0.8022058841221206), (2.0, 0.8022058841221206)]
fuzzy :  None
pembilang :  22.412092319788993
penyebut :  6.410683349794905
rata-rata tertimbang :  3.496053555742387
 
[0.0, 0.0, 2.0, 11.0, 

[11.0, 14.0, 14.0, 0.0, 0.0, 9.0, 14.0, 2.0, 0.0]
mean :  7.111111111111111
standar deviasi :  6.136311676215145
fuzzy :  [(11.0, 0.8180769709503817), (14.0, 0.5325392919942844), (14.0, 0.5325392919942844), (0.0, 0.5109897291977897), (0.0, 0.5109897291977897), (9.0, 0.9537324124951829), (14.0, 0.5325392919942844), (2.0, 0.7069118664392984), (0.0, 0.5109897291977897)]
fuzzy :  None
pembilang :  41.362912389549386
penyebut :  5.609308313461086
rata-rata tertimbang :  7.37397733875452
 
[4.0, 2.0, 2.0, 2.0, 2.0, 0.0, 11.0, 16.0, 2.48115114299134]
mean :  4.609016793665704
standar deviasi :  4.988528938761348
fuzzy :  [(4.0, 0.9925762819530651), (2.0, 0.8721864897139282), (2.0, 0.8721864897139282), (2.0, 0.8721864897139282), (2.0, 0.8721864897139282), (0.0, 0.6526119181441051), (11.0, 0.4401809310051256), (16.0, 0.07377258182371807), (2.48115114299134, 0.9130508021470596)]
fuzzy :  None
pembilang :  19.235565637115894
penyebut :  6.560938473928786
rata-rata tertimbang :  2.9318314313649334

[0.0, 11.0, 10.0, 11.0, 2.0, 11.0, 4.0, 0.0, 11.0]
mean :  6.666666666666667
standar deviasi :  4.760952285695234
fuzzy :  [(0.0, 0.3752020847045813), (11.0, 0.6608867807328891), (10.0, 0.7826476952899146), (11.0, 0.6608867807328891), (2.0, 0.618571588960075), (11.0, 0.6608867807328891), (4.0, 0.8548352822891042), (0.0, 0.3752020847045813), (11.0, 0.6608867807328891)]
fuzzy :  None
pembilang :  41.56197961222283
penyebut :  5.650005858879814
rata-rata tertimbang :  7.356094958185234
 
[0.0, 0.0, 23.0, 40.0, 3.0, 6.515531096158945, 16.0, 16.0, 0.0]
mean :  11.612836788462106
standar deviasi :  12.818087658486888
fuzzy :  [(0.0, 0.663417571333332), (0.0, 0.663417571333332), (23.0, 0.6739786460178584), (40.0, 0.08612018881406602), (3.0, 0.7979411243242615), (6.515531096158945, 0.9239839711618848), (16.0, 0.9431159268659759), (16.0, 0.9431159268659759), (0.0, 0.663417571333332)]
fuzzy :  None
pembilang :  57.54009574011509
penyebut :  6.358508498050018
rata-rata tertimbang :  9.04930704390

[0.0, 16.0, 23.0, 26.0, 4.0, 0.0, 9.049307043902054, 23.0, 6.515531096158945]
mean :  11.951648682229
standar deviasi :  9.688280133055343
fuzzy :  [(0.0, 0.4672787479627706), (16.0, 0.9164069708733449), (23.0, 0.5219573322212084), (26.0, 0.3495206444854862), (4.0, 0.7140660986545017), (0.0, 0.4672787479627706), (9.049307043902054, 0.9561244769315335), (23.0, 0.5219573322212084), (6.515531096158945, 0.8543608356641608)]
fuzzy :  None
pembilang :  64.83522852344363
penyebut :  5.768951186976985
rata-rata tertimbang :  11.238650912804523
 
[0.0, 16.0, 14.0, 11.0, 11.0, 2.0, 4.0, 21.0, 0.0]
mean :  8.777777777777779
standar deviasi :  7.161281353032008
fuzzy :  [(0.0, 0.4718350753350057), (16.0, 0.6014010565815775), (14.0, 0.7665458753795286), (11.0, 0.9529990492869519), (11.0, 0.9529990492869519), (2.0, 0.6390100342545272), (4.0, 0.8004881928581556), (21.0, 0.2331026111652689), (0.0, 0.4718350753350057)]
fuzzy :  None
pembilang :  50.69516591934391
penyebut :  5.890216019482973
rata-rata

...

[89.0, 51.0, 28.0, 64.0, 58.0, 22.0, 16.0, 38.0, 45.0]
mean :  45.666666666666664
standar deviasi :  21.61789382278795
fuzzy :  [(89.0, 0.13414601514610167), (51.0, 0.9700287587048094), (28.0, 0.7161303330058768), (64.0, 0.6979777592199926), (58.0, 0.8498233067528299), (22.0, 0.5492504087414813), (16.0, 0.390029316954924), (38.0, 0.9390563109551406), (45.0, 0.9995246511426005)]
fuzzy :  None
pembilang :  274.4091669291603
penyebut :  6.245966860623757
rata-rata tertimbang :  43.93381730202716
 
[0.0, 0.0, 0.0, 4.0, 4.0, 9.0, 11.0, 19.0, 21.0]
mean :  7.555555555555555
standar deviasi :  7.617394000445605
fuzzy :  [(0.0, 0.6114855896066916), (0.0, 0.6114855896066916), (0.0, 0.6114855896066916), (4.0, 0.8967977866813373), (4.0, 0.8967977866813373), (9.0, 0.982183760521532), (11.0, 0.9028277990372882), (19.0, 0.3235181694340912), (21.0, 0.21068488501897356)]
fuzzy :  None
pembilang :  36.516369732200836
penyebut :  6.047266956194634
rata-rata tertimbang :  6.038491436994458
 
[0.0, 0.0, 4

[0.0, 1.932660665919216, 11.0, 9.0, 0.0, 4.0, 0.0, 0.0, 0.0]
mean :  2.8814067406576904
standar deviasi :  4.041625553105295
fuzzy :  [(0.0, 0.7756063887363334), (1.932660665919216, 0.9728265828468827), (11.0, 0.1330117078004926), (9.0, 0.3179634230272762), (0.0, 0.7756063887363334), (4.0, 0.9624276971554587), (0.0, 0.7756063887363334), (0.0, 0.7756063887363334), (0.0, 0.7756063887363334)]
fuzzy :  None
pembilang :  10.05465405310151
penyebut :  6.264261354511778
rata-rata tertimbang :  1.6050821452172868
 
[0.0, 41.0, 48.0, 72.0, 59.0, 52.0, 26.0, 46.0, 50.0]
mean :  43.77777777777778
standar deviasi :  19.42951808047503
fuzzy :  [(0.0, 0.07901656815888497), (41.0, 0.9898333273731299), (48.0, 0.9766672202397887), (72.0, 0.34825090056778746), (59.0, 0.7357448915046279), (52.0, 0.9143588073190552), (26.0, 0.6579948288375196), (46.0, 0.9934813670943349), (50.0, 0.9500189185637908)]
fuzzy :  None
pembilang :  313.80181877835724
penyebut :  6.645366829658919
rata-rata tertimbang :  47.2211

[0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 2.0, 21.0, 2.0]
mean :  3.2222222222222223
standar deviasi :  6.355710953811152
fuzzy :  [(0.0, 0.8794123796283898), (0.0, 0.8794123796283898), (0.0, 0.8794123796283898), (2.0, 0.9816815573481376), (2.0, 0.9816815573481376), (0.0, 0.8794123796283898), (2.0, 0.9816815573481376), (21.0, 0.020008965532331544), (2.0, 0.9816815573481376)]
fuzzy :  None
pembilang :  8.273640734964063
penyebut :  7.46438471343844
rata-rata tertimbang :  1.1084156367327487
 
[0.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
mean :  14.222222222222221
standar deviasi :  5.028314888437673
fuzzy :  [(0.0, 0.018323236638827504), (16.0, 0.939419150480615), (16.0, 0.939419150480615), (16.0, 0.939419150480615), (16.0, 0.939419150480615), (16.0, 0.939419150480615), (16.0, 0.939419150480615), (16.0, 0.939419150480615), (16.0, 0.939419150480615)]
fuzzy :  None
pembilang :  120.2456512615187
penyebut :  7.533676440483746
rata-rata tertimbang :  15.96108516359346
 
[0.0, 1.10841563673274

[0.0, 34.0, 40.0, 46.0, 38.0, 26.0, 39.0, 50.0, 26.0]
mean :  33.22222222222222
standar deviasi :  13.966450099973928
fuzzy :  [(0.0, 0.05908062563657222), (34.0, 0.9984507292257991), (40.0, 0.8889262943429673), (46.0, 0.6580533545196654), (38.0, 0.9431720235687469), (26.0, 0.8748623941712661), (39.0, 0.9179973748286965), (50.0, 0.4860344273314475), (26.0, 0.8748623941712661)]
fuzzy :  None
pembilang :  241.2118312527102
penyebut :  6.701439617796428
rata-rata tertimbang :  35.99403188117147
 
[0.0, 4.0, 4.0, 16.0, 18.0, 26.0, 41.0, 43.0, 46.0]
mean :  22.0
standar deviasi :  16.924669699714805
fuzzy :  [(0.0, 0.42966331504098115), (4.0, 0.5680782791028304), (4.0, 0.5680782791028304), (16.0, 0.9391003890075807), (18.0, 0.9724605877138113), (26.0, 0.9724605877138113), (41.0, 0.5325522268775555), (43.0, 0.46315062076103464), (46.0, 0.3659252069734581)]
fuzzy :  None
pembilang :  120.94117583183497
penyebut :  5.811469492293893
rata-rata tertimbang :  20.810773590432678
 
[0.0, 28.0, 35.0

[0.0, 77.0, 87.0, 64.0, 57.0, 73.0, 89.0, 70.0, 98.0]
mean :  68.33333333333333
standar deviasi :  27.022624266014986
fuzzy :  [(0.0, 0.04088693042411065), (77.0, 0.9498749089057017), (87.0, 0.7877588916115407), (64.0, 0.9872260440197863), (57.0, 0.9158160483112588), (73.0, 0.9852004059465477), (89.0, 0.7464530873626781), (70.0, 0.998099995984609), (98.0, 0.5474030578295566)]
fuzzy :  None
pembilang :  518.9258269225467
penyebut :  6.958719370395789
rata-rata tertimbang :  74.57202960794665
 
[0.0, 0.0, 13.462290980253753, 14.0, 14.0, 14.0, 21.0, 35.0, 35.0]
mean :  16.273587886694862
standar deviasi :  11.910622308973645
fuzzy :  [(0.0, 0.39325262257735216), (0.0, 0.39325262257735216), (13.462290980253753, 0.9725314733085586), (14.0, 0.9819478152747818), (14.0, 0.9819478152747818), (14.0, 0.9819478152747818), (21.0, 0.9242929695437204), (35.0, 0.2905881063785175), (35.0, 0.2905881063785175)]
fuzzy :  None
pembilang :  94.0856297295899
penyebut :  6.210349346588363
rata-rata tertimbang

In [15]:
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2.000000,50.000000,7797.609984,98.000000
1,4.578596,13.000000,3250.000000,28.000000
2,1.000000,16.000000,4000.000000,71.165210
3,2.000000,20.000000,5000.000000,45.000000
4,1.000000,24.000000,6000.000000,77.000000
...,...,...,...,...
743,23.000000,2.000000,500.000000,22.346593
744,9.061233,2.000000,721.589339,52.000000
745,6.645310,5.828262,1409.971487,62.000000
746,39.000000,1.000000,250.000000,24.225557


In [16]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
count,748.000000,748.000000,748.000000,748.000000
mean,9.129100,5.506494,1348.279747,33.752015
std,7.755012,5.841497,1399.205281,24.537291
min,1.000000,0.808930,152.548052,0.579622
25%,3.000000,2.000000,500.000000,14.000000
50%,6.926253,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,49.000000
max,74.000000,50.000000,11500.000000,98.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [52]:
x = dummy.values
y = df.iloc[:, -1].values

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [54]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [55]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[128  18]
 [ 29  12]]
0.7486631016042781
